# Table of Contents:
* [I. Import libraries](#I.-Import-libraries)
* [II. Data Wangling](#II.-Data-Wangling)
    * [Prepare the dataframe for business](#Prepare-the-dataframe-for-business)
    * [Prepare the dataframe for users](#Prepare-the-dataframe-for-users)
    * [Prepare the dataframe for reviews](#Prepare-the-dataframe-for-reviews)
    * [Prepare categories of businesses](#Prepare-categories-of-businesses)
    * [Add category labels to business dataframe](#Add-category-labels-to-business-dataframe)
    * [Prepare review dataframe with only selected users](#Prepare-review-dataframe-with-only-selected-users)


# I. Import libraries

In [1]:
import json
import math
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# II. Data Wangling
### Prepare the dataframe for business

The original business file is a `csv` file of 148 MB. However, only some attributes are useful. So useless columns are removed from the data. The new dataframe is stored as a csv named df_user.csv. This file has 46.8 MB.

In [2]:
# Read original dataframe
df_business_ori = pd.read_json("./data/yelp_academic_dataset_business.json", lines = True)

In [3]:
# Keep only relevant colunms
df_business = df_business_ori[['business_id','state','latitude','longitude', 'categories','stars']].copy()
# Show an example
df_business[:3]

,business_id,state,latitude,longitude,categories,stars
0,f9NumwFMBDn751xgFiRbNA,NC,35.462724,-80.852612,"Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...",3.5
1,Yzvjg0SayhoZgCljUJRF9Q,AZ,33.569404,-111.890264,"Health & Medical, Fitness & Instruction, Yoga,...",5.0
2,XNoUzKckATkOD1hP6vghZg,QC,45.479984,-73.580070,"Pets, Pet Services, Pet Groomers",5.0


In [4]:
# Save smaller dataframe as csv in order to not load the entire dataset next time
df_business.to_csv('./data/df_business.csv',index=False)

### Prepare the dataframe for users
The original user file is a `json` file of 3.3 GB. However, we only use attributes `user_id` and `firends`. So a new dataframe for users is created and stored as a `csv` named `df_user.csv`. This file has 2.5 GB.

In [5]:
# Read large json file of user
with open("./data/yelp_academic_dataset_user.json") as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

In [6]:
# Convert to DataFrame
df_user_ori = pd.DataFrame(data)    
# Keep only relevant colunms
df_users = df_user_ori[['user_id','friends']].copy()

In [7]:
# Show an example
df_users[:3]

,user_id,friends
0,ntlvfPzc8eglqvk92iDIAw,"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg..."
1,FOBRPlBHa3WPHFB5qYDlVg,"ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOug..."
2,zZUnPeh2hEp0WydbAZEOOg,"Uwlk0txjQBPw_JhHsQnyeg, Ybxr1tSCkv3lYA0I1qmnPQ..."


In [8]:
# Save smaller dataframe as csv in order to not load the entire dataset next time
df_users.to_csv('./data/df_user.csv',index=False)

### Prepare the dataframe for reviews

The original review file is a `json` file of 6.3 GB containing many useless attributes. We have used `jq`, a tool which allows us to select the desired attribute. The result is stored in a `csv` file named `df_reviews.csv`. This file has 729.9 MB.

In [9]:
# Read the review file
df_review = pd.read_csv("./data/df_reviews.csv")
# Show an example
df_review.head()

,review_id,user_id,business_id,stars,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,2017-01-14 21:56:57


### Prepare categories of businesses

The categories used to describe the businesses were way too specific, so we decided to label those 1336 very sharp categories to 17 more general categories described in the `business_category-list.pdf`, making it easier to make some analysis around those 17 labels. Since this labelling was done by hand, we had to correct a few of the typos (missing letters, lower cases, additional spaces). The 17 more general categories are `Automotive`, `BusinessSupport`, `Computer(& Elecetronics)`, `Construction`, `Education`, `Entertainment`, `Food(& Dining)`, `Health`, `Home(& Garden)`, `Legal(& Finance)`, `Manufacturing`, `Merchant`, `Miscellaneous`, `PersonalCare`, `RealEstate`, `Sport`, `Travel(& Transportation)`.

In [10]:
# Read prepared csv mapping category name to category label
df_categories = pd.read_csv('name_category.csv')

In [11]:
def correction_name(x):
    """
        Sanitize category name
        x: row of the dataframe
        Return: sanitized category name
    """
    name = x[0]
    name = name.replace("'", "")
    return name

def correction_category(x):
    """
        Sanitize category label
        x: row of the dataframe
        Return: sanitized category
    """
    category = x[1]
    category = category.replace(" ", "")
    category = category.replace("Consruction", "Construction")
    category = category.replace("Constructor", "Construction")
    category = category.replace("Entertainement", "Entertainment")
    category = category.replace("Entertainemnt", "Entertainment")
    category = category.replace("Entertainments", "Entertainment")
    category = category.replace("Merchants", "Merchant")
    category = category.replace("'", "")
    category = category.replace("Mischellaneous", "Miscellaneous")
    category = category.replace("food", "Food")
    return category

In [12]:
# Sanitize category name and category label
df_categories['name'] = df_categories.apply(lambda x: correction_name(x), axis=1)
df_categories['category'] = df_categories.apply(lambda x: correction_category(x), axis=1)

In [13]:
# Save sanitized file
df_categories.to_csv('name_category.csv', index=False)

### Add category labels to business dataframe

In [14]:
# Add category labels to business dataframe
df_business['categories'] = df_business.categories.apply(lambda x: [] if x == None else x.split(', '))

In [15]:
# Create a dictionary mapping category name to category label
dict_categories = df_categories.set_index('name')['category'].to_dict()
# Tokenize the category label and store in a dictionary
indexCategory = {val: key for key, val in enumerate(set(df_categories['category']))}

In [16]:
def create_category_mask(row): 
    """
        For categories of each business, create a mask of categories
        row: row of the dataframe
        Return: mask of categories. If 1 at index i, it means that the business have i-th category.
    """
    res  = np.zeros(numberCat)
    for r in row:
        category = dict_categories.get(r)
        index = indexCategory.get(category)
        res[index] = 1
    return res > 0
# Vectorize the function 
#create_category_mask_vect = np.vectorize(test, signature='()->(k)')

In [17]:
# Fetch the number of categories
numberCat = len(set(df_categories['category']))
# Create category mask for each business in reviews
categoryMask = df_business.apply(lambda row: create_category_mask(row.categories),axis=1)

In [18]:
# For each category, add a new column to the business dataframe
for category_name in set(df_categories['category']):
    # Create "category_name" column, and assign the category mask
    # If mask at row i is True, it means that the business at row i has category: "category_name"
    df_business[category_name] = [tup[indexCategory.get(category_name)] for tup in categoryMask]

In [19]:
# Show an example
df_business.head(2)

,business_id,state,latitude,longitude,categories,stars,Construction,Computers,Home,Merchant,...,Education,Legal,BusinessSupport,PersonalCare,Entertainment,RealEstate,Automotive,Food,Miscellaneous,Health
0,f9NumwFMBDn751xgFiRbNA,NC,35.462724,-80.852612,"[Active Life, Gun/Rifle Ranges, Guns & Ammo, S...",3.5,False,False,False,True,...,False,False,False,True,True,False,False,False,False,False
1,Yzvjg0SayhoZgCljUJRF9Q,AZ,33.569404,-111.890264,"[Health & Medical, Fitness & Instruction, Yoga...",5.0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True


In [20]:
# Save smaller dataframe as csv in order to not load the entire dataset next time
df_business.to_csv('./data/df_business.csv',index=False)

### Prepare review dataframe with only selected users
For our study, we need to infer user's home, so we need **at least 3 reviews**. Also, we will study users with **at least 3 friends**. 

In [21]:
# Convert the string of friends to a list
df_users['friend_list'] = df_users.apply(lambda x: x.friends.split(', '), axis=1)
df_users = df_users.drop('friends', axis=1)

In [23]:
# We count the number of reviewe per user
df_numberOfReviews = df_review.groupby('user_id').count().drop(['business_id', 'stars', 'date'], axis = 1).rename(columns={'review_id': 'review_count'})

In [24]:
# We merge the df_users dataset to the df_numberOfReviews dataset
df_users = pd.merge(df_users, df_numberOfReviews, left_on='user_id', right_on='user_id', how='inner')

In [25]:
# We create a mask for users with more than 3 friends
mask = df_users.apply(lambda x: len(x.friend_list) >= 3, axis=1)

In [26]:
print('Number of users with more than 3 reviews:', df_numberOfReviews[df_numberOfReviews['review_count'] >= 3].shape[0])
print('Number of users with more than 3 friends:', df_users[mask].shape[0])

Number of users with more than 3 reviews: 602736
Number of users with more than 3 friends: 957463


In [27]:
# Select users that satisfy both conditions
df_selectedUsers = df_users[mask&(df_users['review_count'] >= 3)]
print('Number of users with more than 3 reviews and with more than 3 friends:', df_selectedUsers.shape[0])

Number of users with more than 4 reviews and with more than 3 friends: 343424


In [37]:
# Save smaller user dataframe as csv in order to not load the entire dataset next time
df_selectedUsers.to_csv('./data/selectedUsers.csv',index=False)

In [28]:
# Number of checkins of the selected users
print('Number of checkins of selected users:', df_selectedUsers['review_count'].sum())

Number of checkins of selected users: 4516345


Prepare a dataframe containly only selected users.

In [29]:
# Merge review and business dataframe
df_review = pd.merge(df_review, df_business, left_on='business_id', right_on='business_id', how='inner').drop(['state'], axis=1)

In [33]:
# Select only rows of selected users
df_selectedReviews = df_review[df_review["user_id"].isin(df_selectedUsers.user_id)]

In [34]:
# Convert date to datetime object
df_selectedReviews["date"] = pd.to_datetime(df_selectedReviews.date.values)

<ipython-input-34-f0a7b85b7e69>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selectedReviews["date"] = pd.to_datetime(df_selectedReviews.date.values)


In [40]:
# Show an example
df_selectedReviews.head()

,review_id,user_id,business_id,stars_x,date,latitude,longitude,categories,stars_y,Construction,...,Education,Legal,BusinessSupport,PersonalCare,Entertainment,RealEstate,Automotive,Food,Miscellaneous,Health
1,t7xOZF5UKXjSpVcXLOSAgw,owbC7FP8SNAlwv6f9S5Stw,-MhfebM0QIsKt87iDN-FNw,2,2014-03-14 08:24:25,36.112896,-115.177637,"[Shopping, Arts & Entertainment, Art Galleries...",3.5,False,...,False,False,False,False,True,False,False,False,False,False
2,MimB5Xh85rG7phUMPrShag,v9vGnjphb0Hta0lvtf5haA,-MhfebM0QIsKt87iDN-FNw,3,2015-10-07 22:16:59,36.112896,-115.177637,"[Shopping, Arts & Entertainment, Art Galleries...",3.5,False,...,False,False,False,False,True,False,False,False,False,False
4,cnV5xtm6WuyaLfot9uWbDg,LkWNo83Lg92C5V4JEyxOZA,-MhfebM0QIsKt87iDN-FNw,3,2010-10-10 01:27:31,36.112896,-115.177637,"[Shopping, Arts & Entertainment, Art Galleries...",3.5,False,...,False,False,False,False,True,False,False,False,False,False
5,i593z2rGxk5Lj23Bx9o1Lg,mLtaUzSjIFO_3BmfPNUKqg,-MhfebM0QIsKt87iDN-FNw,1,2015-08-20 01:10:53,36.112896,-115.177637,"[Shopping, Arts & Entertainment, Art Galleries...",3.5,False,...,False,False,False,False,True,False,False,False,False,False
6,TWpbq_vbpJRotCQ6l8SeXA,wKX1tAeRIYPU4NtM-R5N2w,-MhfebM0QIsKt87iDN-FNw,5,2012-03-31 22:00:15,36.112896,-115.177637,"[Shopping, Arts & Entertainment, Art Galleries...",3.5,False,...,False,False,False,False,True,False,False,False,False,False


In [36]:
# Save smaller dataframe as csv in order to not load the entire dataset next time
df_selectedReviews.to_csv('./data/selectedReviews.csv',index=False)